In [210]:
# import libraries :) eventually there should be a .conda environment in the folder for sharing any downloaded packages
import pandas as pd
import networkx as nx
from pyvis.network import Network

In [211]:
# now let's iterate through our amicus briefs and store the lexisCite, position, and amicus name
amicus_briefs_csv = pd.read_csv('amicus_briefs.csv', encoding = "latin")
amicus_briefs_csv = amicus_briefs_csv[['LexisCite', 'Position', 'AmicusName']]
amicus_briefs_csv.head(5)

,LexisCite,Position,AmicusName
0,1953 U.S. LEXIS 2673,1,SECURITIES AND EXCHANGE COMMISSION
1,1953 U.S. LEXIS 2680,2,BOSTON AMERICAN LEAGUE BASE BALL COMPANY
2,1954 U.S. LEXIS 2094,1,AMERICAN FEDERATION OF TEACHERS
3,1954 U.S. LEXIS 2094,1,UNITED STATES
4,1954 U.S. LEXIS 2094,1,AMERICAN JEWISH CONGRESS


In [212]:
# let's now get the justices and their votes for each case (identified by LexisCite and caseName)

justices_csv = pd.read_csv('justices.csv', encoding='latin')
justices_csv = justices_csv[['lexisCite', 'justiceName', 'vote', 'caseName', 'caseDisposition']]
justices_csv.head(5)

/var/folders/_3/sy3t66qx59z0h8k2_18_hldh0000gn/T/ipykernel_15615/1664479359.py:3: DtypeWarning: Columns (6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  justices_csv = pd.read_csv('justices.csv', encoding='latin')


,lexisCite,justiceName,vote,caseName,caseDisposition
0,1946 U.S. LEXIS 1724,HHBurton,2.0,HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...,3.0
1,1946 U.S. LEXIS 1724,RHJackson,1.0,HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...,3.0
2,1946 U.S. LEXIS 1724,WODouglas,1.0,HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...,3.0
3,1946 U.S. LEXIS 1724,FFrankfurter,4.0,HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...,3.0
4,1946 U.S. LEXIS 1724,SFReed,1.0,HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...,3.0


In [213]:
# for simplicity, we want the final form factor to be a dictionary as it's easier to parse and find information
# let's first convert the amicus briefs into a dictionary with one key per docket
# each docket will have all the amicus briefs with the organization names and their positions

amici_data = {}

for _, brief_row in amicus_briefs_csv.iterrows():
    lexis_cite = brief_row['LexisCite']
    amicus_name = brief_row['AmicusName']
    position = brief_row['Position']
    
    if lexis_cite not in amici_data:
        amici_data[lexis_cite] = []

    amici_data[lexis_cite].append((amicus_name, position))

amici_data

{'1953 U.S. LEXIS 2673': [('SECURITIES AND EXCHANGE COMMISSION', 1)],
 '1953 U.S. LEXIS 2680': [('BOSTON AMERICAN LEAGUE BASE BALL COMPANY', 2)],
 '1954 U.S. LEXIS 2094': [('AMERICAN FEDERATION OF TEACHERS', 1),
  ('UNITED STATES', 1),
  ('AMERICAN JEWISH CONGRESS', 1),
  ('AMERICAN VETERANS COMMITTEE', 1),
  ('AMERICAN ETHICAL UNION', 1),
  ('AMERICAN JEWISH COMMITTEE', 1),
  ('ANTI-DEFAMATION LEAGUE', 1),
  ('JAPANESE AMERICAN CITIZENS LEAGUE', 1),
  ('UNITARIAN FELLOWSHIP FOR SOCIAL JUSTICE', 1),
  ('AMERICAN CIVIL LIBERTIES UNION', 1),
  ('CONGRESS OF INDUSTRIAL ORGANIZATIONS', 1)],
 '1954 U.S. LEXIS 2095': [('AMERICAN FEDERATION OF TEACHERS', 1),
  ('UNITED STATES', 1),
  ('AMERICAN VETERANS COMMITTEE', 1),
  ('AMERICAN JEWISH COMMITTEE, WASHINGTON CHAPTER', 1),
  ('AMERICAN JEWISH CONGRESS COMMISSION ON LAW AND SOCIAL ACTION, WASHINGTON CHAPTER',
   1),
  ('AMERICANS FOR DEMOCRATIC ACTION, WASHINGTON CHAPTER', 1),
  ('CATHOLIC INTERRACIAL COUNCIL OF WASHINGTON', 1),
  ('COMMISSIO

In [214]:
# let's also do this for the judges

justices_data = {}

for _, justice_row in justices_csv.iterrows():
    lexis_cite = justice_row['lexisCite']
    justice_name = justice_row['justiceName']
    case_name = justice_row['caseName']
    vote = justice_row['vote']
    disposition = justice_row['caseDisposition']
    
    if lexis_cite not in justices_data:
        justices_data[lexis_cite] = {
            'justices':[],
            'caseName': case_name,
            'decision': disposition
        }

    justices_data[lexis_cite]['justices'].append((justice_name, vote))

justices_data

{'1946 U.S. LEXIS 1724': {'justices': [('HHBurton', 2.0),
   ('RHJackson', 1.0),
   ('WODouglas', 1.0),
   ('FFrankfurter', 4.0),
   ('SFReed', 1.0),
   ('HLBlack', 1.0),
   ('WBRutledge', 1.0),
   ('FMurphy', 1.0),
   ('FMVinson', 1.0)],
  'caseName': 'HALLIBURTON OIL WELL CEMENTING CO. v. WALKER et al., DOING BUSINESS AS DEPTHOGRAPH CO.',
  'decision': 3.0},
 '1946 U.S. LEXIS 1725': {'justices': [('HHBurton', 1.0),
   ('RHJackson', 2.0),
   ('WODouglas', 1.0),
   ('FFrankfurter', 1.0),
   ('SFReed', 1.0),
   ('HLBlack', 2.0),
   ('WBRutledge', 4.0),
   ('FMurphy', 2.0),
   ('FMVinson', 1.0)],
  'caseName': 'CLEVELAND v. UNITED STATES',
  'decision': 2.0},
 '1946 U.S. LEXIS 3037': {'justices': [('HHBurton', 2.0),
   ('RHJackson', 1.0),
   ('WODouglas', 2.0),
   ('FFrankfurter', 2.0),
   ('SFReed', 2.0),
   ('HLBlack', 1.0),
   ('WBRutledge', 1.0),
   ('FMurphy', 1.0),
   ('FMVinson', 1.0)],
  'caseName': 'CHAMPLIN REFINING CO. v. UNITED STATES ET AL.',
  'decision': 2.0},
 '1946 U.S. 

In [215]:
# Now we can mereg our dictionaries together to get a final pre-processed dictionary

preproc_data = {}

for lexis_cite, brief in amici_data.items():
    justices_found = justices_data[lexis_cite]

    # if we can't find the corresponding case, we'll skip this row
    if justices_found is None:
        continue

    preproc_data[lexis_cite] = {
        'case_name': justices_found['caseName'],
        'justices': justices_found['justices'],
        'case_decision': justices_found['decision'],
        'amici': brief
    }

preproc_data

{'1953 U.S. LEXIS 2673': {'case_name': 'WILKO v. SWAN et al., DOING BUSINESS AS HAYDEN, STONE & CO., et al.',
  'justices': [('HLBlack', 1.0),
   ('WODouglas', 1.0),
   ('EWarren', 1.0),
   ('TCClark', 1.0),
   ('FFrankfurter', 2.0),
   ('HHBurton', 1.0),
   ('SFReed', 1.0),
   ('SMinton', 2.0),
   ('RHJackson', 3.0)],
  'case_decision': 3.0,
  'amici': [('SECURITIES AND EXCHANGE COMMISSION', 1)]},
 '1953 U.S. LEXIS 2680': {'case_name': 'TOOLSON v. NEW YORK YANKEES, INC. et al.',
  'justices': [('HLBlack', 1.0),
   ('WODouglas', 1.0),
   ('EWarren', 1.0),
   ('TCClark', 1.0),
   ('FFrankfurter', 1.0),
   ('HHBurton', 2.0),
   ('SFReed', 2.0),
   ('SMinton', 1.0),
   ('RHJackson', 1.0)],
  'case_decision': 2.0,
  'amici': [('BOSTON AMERICAN LEAGUE BASE BALL COMPANY', 2)]},
 '1954 U.S. LEXIS 2094': {'case_name': 'BROWN et al. v. BOARD OF EDUCATION OF TOPEKA et al.',
  'justices': [('HLBlack', 1.0),
   ('WODouglas', 1.0),
   ('EWarren', 1.0),
   ('TCClark', 1.0),
   ('FFrankfurter', 1.0),

In [216]:
# let's make a function to determine if the judges agreed with the amicus brief
# here we need to determine whether the judge followed the amicus decision

# for amicus brief position:
    # Indicates the position on the case expressed by the amicus brief. = 1 if the brief
    # argues the lower court decision should be reversed, = 2 if lower court decision
    # should be affirmed, = 3 if the brief explicitly does not adopt a position or the
    # position is unclear.

# for final case decision:
    # 1	stay, petition, or motion granted
    # 2	affirmed (includes modified)
    # 3	reversed
    # 4	reversed and remanded
    # 5	vacated and remanded
    # 6	affirmed and reversed (or vacated) in part
    # 7	affirmed and reversed (or vacated) in part and remanded
    # 8	vacated
    # 9	petition denied or appeal dismissed
    # 10	certification to or from a lower court
    # 11	no disposition

# for justice vote:
    # 1	voted with majority or plurality
    # 2	dissent
    # 3	regular concurrence
    # 4	special concurrence
    # 5	judgment of the Court
    # 6	dissent from a denial or dismissal of certiorari , or dissent from summary affirmation of an appeal
    # 7	jurisdictional dissent
    # 8	justice participated in an equally divided vote

def agreedWithAmicus(brief_position, case_decision, justice_vote):
    # if we have any invalid values, let's remove this scenario
    try:
        position, decision, vote = int(brief_position), int(case_decision), int(justice_vote)
    except:
        return 2
    
    # agrees with amicus brief for reversing lower court decision
    if position == 1 and (decision == 3 or decision == 4) and vote == 1:
        return 1
    
    # disagrees with amicus brief for reversing lower court decision
    if position == 1 and (decision == 3 or decision == 4) and vote == 2:
        return 0

    # agrees with amicus brief for affirming lower court decision
    if position == 2 and (decision == 2) and vote == 1:
        return 1
    
    # disagrees with amicus brief for affirming lower court decision
    if position == 2 and (decision == 2) and vote == 2:
        return 0

    # invalid scenario
    return 2

In [217]:
# Let's construct a graph now
# We make an assumption that the judges and amicus organizations have no overlapping names

G = nx.MultiDiGraph()

# going to use a counter so we can visualize faster
maxCount = 200 # Comment line out for full graph
counter = 0

for lexis_cite, docket in preproc_data.items():
    if counter >= maxCount:
        break         
    for amicus in docket['amici']:
        for justice in docket['justices']:
            # 0 = disagree, 1 = agree, 2 = trash
            agreement = agreedWithAmicus(amicus[1], docket['case_decision'], justice[1])
            if agreement < 2:
                G.add_edge(justice[0], amicus[0], caseName=docket['case_name'], agreement=agreement)
                counter += 1

In [227]:
# Create a visualization of the graph

net = Network(
    directed = True,
    select_menu = True, # Show part 1 in the plot (optional)
    filter_menu = True, # Show part 2 in the plot (optional)
)
net.show_buttons() # Show part 3 in the plot (optional)
net.from_nx(G) # Create directly from nx graph
net.show('test.html', notebook=False)
print("open the test.html file created!!")

test.html
open the test.html file created!!


In [226]:
# Quick sanity check
affirmed = 0
reversed = 0
for edge in G.edges(data=True):
    if edge[2]['agreement']:
        affirmed += 1
    else:
        reversed += 1

print("affirmed:", affirmed, "\nreversed:", reversed, "\ntotal:", reversed+affirmed)

affirmed: 177 
reversed: 26 
total: 203
